# Generación de nombres

esta es una libreta para generar nombres de municipios usando una red recurrente 
hecha solo con numpy obtenida de [este gist](https://gist.github.com/karpathy/d4dee566867f8291f086).

### Apertura de archivo

In [25]:
## Importamos lo necesario que es solo numpy
import numpy as np

# abrimos el archivo con los datos
data = open('municipios.txt', 'r').read() # puede ser texto plano .txt
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print ('el documento tiene %d caracteres, %d unicos.' % (data_size, vocab_size))
print(chars)
print(data[:100])
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

el documento tiene 35529 caracteres, 68 unicos.
['S', 'K', 'Á', 'ñ', 'I', '\n', 'y', 'A', 'T', 'Y', 'r', 'é', 'G', 'á', 'H', '8', 'd', 'o', 't', ' ', 's', 'l', 'n', 'X', 'ú', 'q', 'C', 'P', 'O', 'm', 'p', 'L', 'U', '-', '6', 'J', '0', 'u', 'x', 'j', ',', 'ó', 'E', 'M', 'F', 'g', 'N', 'z', 'f', 'Z', 'ü', 'í', 'V', 'k', 'i', 'h', 'v', '2', 'B', 'a', '.', 'Ñ', 'D', 'R', 'e', 'Q', 'c', 'b']
Abalá
Abasolo
Abasolo
Abasolo
Abasolo
Abejones
Acacoyagua
Acajete
Acajete
Acala
Acámbaro
Acambay de 


### Hiperparametros 


In [11]:
# Declaracion de hiperparametros
hidden_size = 100 # tamaño de las capas ocultas 
seq_length = 25 # numero de pasos para desenrrollar por RNN
learning_rate = 1e-1

# parametros del modelo
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # entrada a ocultas
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # ocultas a ocultas
Why = np.random.randn(vocab_size, hidden_size)*0.01 # ocultas a salida
bh = np.zeros((hidden_size, 1)) # ocultas bias
by = np.zeros((vocab_size, 1)) # salida bias

### Función de perdida

In [19]:
def lossFun(inputs, targets, hprev):
  """
  entradas y objetivos son ambos listas de enteros.
  hprev es Hx1 arreglo de los estados ocultos iniciales
  regresa la perdida, gradientes en parametros del modelo, y el ultimo estado oculto
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

### Función muestra

In [17]:
def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

### Generación de caracteres

In [28]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 200 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print ('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 200 == 0: print ('iter %d, loss: %f' % (n, smooth_loss)) # Imprimir el progreso
  
  # realizar la actualizacion de parametros con Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # actualizar adagrad 

  p += seq_length # mueve el puntero de datos
  n += 1 # contador de iteraciones

----
 blas
Kan
Jutac
La Aquilmpncisto
Jachranaz
La Jutlán de Campatla de Mosaca
Lalár
Lo Jimpayo cero
Jamtayacá
Mirilacotlahtepec
La La Lel de Ría Co
Ldro
Leda Igén Lego de Ligoño Cerra
Le Juamaxtepec
Mosac 
----
iter 0, loss: 105.464069
----
 Sazcaquade  on
Codaymhalco llalaa
Conesze de cac
Colitláo Mocenoz
Cacueyquolilo Gotan
Cacarhuecáláonhuepocadalsin en eníeltoyo
Codándezpopánde mcoongen alo delatla cándecato
Cela
Cadan
Codangoya
Cooze 
----
iter 200, loss: 101.177005
----
 ránte Jjás
Imetzánde
Sán
Itlán
Ilatladapapac
Iquistaca Jantlarlac Ralral Ro
Itatlán
Jorzá
IIcen iongo
Itazlpa deztvan
Isvalac
Imatlán
Fra
Isam
Iz Rí
Cos
Jen Fuáa Jalanté
zepadeImo
Iztlacatvacuamorala
 
----
iter 400, loss: 93.703819
----
 e
Pen
Prilcin
Moy
Pesto lo Acarbotrín
Patlo
Pebeñran
Pelotelnixhi
Pixteñatlo Ponco
Pocho
Palmu Aten
Plinegatloqope Emo
Pal
Murares
Pentzijritlántinte Pámántiñis
Perejá
Peratmaljen
PonixtláncitejanPen
 
----
iter 600, loss: 87.255045
----
  Ron
Senal
Jolé
San Juez Rluzas A
San 

----
 un
Turuquelucuando
Tueluman
Teancix
Tutecán
Tuxxtla
Tundan
Tulacuelapa
Tuenatlitla
Tuquis
Tulotte
Tacas
Tuapema
Tlancintlán
TZipan Tuixto
Muaxcuan
Tucuactepec
TCánz
Tuxcuas
Tononto
Mratepeca
Todepe
Tu 
----
iter 7000, loss: 44.158439
----
 ol Azotzos Avuaztipan
Arozoz
Apacaentep
Ayuxgues Ren
Tzaloca de
Midayayayaz
Ayayuelaco
Tulayipa
Axta
Arozon
Ahayóe
Zoteepatío
Axtepec
Amatayolo
Axtolgas Anaya
Ayatla
Allayalo
Arapeyo de Pedepey
Ayunzo 
----
iter 7200, loss: 44.939028
----
 
Faníra
Ala te
San Fluel
Penlinez Brao
Bomonirilc
Joricte
Dzan
Morro Fraleye
Bauin
Gretla de Jeamíralpeno
Fujípa con Fliro
Cuac
Flovine.
Fcunco Flaocoso Fragosco Plimo leo
Frpeco
Frorír
Frannco do
Miy 
----
iter 7400, loss: 45.976308
----
 alanda Taras
Maz Asés
Motepiá
Metededo
Mezhis
Morga
Macía
Macuanríac
Culquas
Matla
Moríro
Mtrapo Terata
Miges
Sanavea
Malano
Flatlanha
Lovatocápec de Latatal
Mones
Tasah
Tinatavepec
Mabaacotla
Mcos
Mi 
----
iter 7600, loss: 46.743364
----
 ngo delapan Cón Yaradetla
Sechurod

----
 
Teacacama
Temaneca
Tezebecotla
Tlakchihuec
Samcapa
Tetarlála
Teviguzumpa
Panamanmes
Tenecona
Tezonco
Teyolla
Temilpa
Tengatla
Tenompan
Temanivis Anco Tlaxine Tergohuinnimingóg
Tartepec
Textea Drongo
 
----
iter 14000, loss: 42.235815
----
 o
Zapalgohuo
Zillintela
Zapareldo
Cacuavalca
Zapaafupaníro
Otlán
Zapadro
Zahuitla
Zarachicillanada
Zetána
Antinigueriacillánitotuluyo
Pajipelates
Vi. Muxpabavo
Lupolipezozuteco Yotlajro
Zalitlán de go 
----
iter 14200, loss: 42.669167
----
  Caatam
Coazamedal Cando
Co
Grolltepec
Cuerro
Coltos
Chiutapec
Chitez Biegamaninitlas
Cuomitlán
Chincalapan de Guihuaugol Id lán
Cullán
Colalo Coayo Laaztía
Cotiltisillo de llla Hiopengo malan Asotdre 
----
iter 14400, loss: 43.720582
----
 
IuIxtro
Flamaxtlas
Hoqal
IuIquamajhimarucuicocajucixtupa
Domdil Ixtán
Izatelo Sanco depancasco
Ixtamegomepabánilálhuana
Ixta
Iíón uma
Helpezasco Imaritepec
Itampalalaso
Freán
Ixtán del Zesíancoalasas 
----
iter 14600, loss: 44.798573
----
 tlán
Puporapo
Pavendes Puztoge

----
 tuelmitla
San Pedel Tohuela
San Suil del Río Melcona
San Pedan
Yanálosco Santo Zablao Elcía la
San Juélega
Solte Cocuollodo
San Pabtín Tuxcendo
San Palvillo
San Pilollinil de lán
Tiogotlán Fentepe Sue 
----
iter 20800, loss: 42.729365
----
  Anilónga
Santo Tiotlán del Periristepec
Santiago Cadado
Santiago Zalacro
Santiaga
Santiago Manquis
Santo Luilpan
Púzaca litanión O
Santa Miñatlán
Santiago Tiaxtitto 0QSiaotín
Santo Lezoño
Santea Tepa 
----
iter 21000, loss: 40.896059
----
 otulerco
Tani
Tlapapa
Toacoyopango
Tinaro
Ta
Tuconacú
Tepingo
Tonquán
Tocotla
Tenapo de Toncán
Tetoguego
Tecalatlán
Tingoha
Tininde Josco
Tana
Tcemapa
TuadoTlanilno
Tusapamtepec
Tochicti
Mizogohupan
T 
----
iter 21200, loss: 41.210518
----
 lphuautlarato
Atlayuezal Ardo
Ataltimo Gringuiyamo
Agatla Atextli. Acompan
Axcilic
Alpa
Acualco
AxtileXo
Atlatetla
Atiucera Catate
ButvCáriniltic el de Alopardo Guitquínim
Ahuapel
Atavanis
Tuleón
Axta 
----
iter 21400, loss: 42.072239
----
 on
Donga
Epape
Elcoha
Escarman

----
 usto
Molarmesuten Bron
Sello
Mocurás de Sorelázeo
Paquosimlán
Penextla Acomcon
Villicuchi
Juinla
Pésactiscitlán
Paloz
Parovos
Pocutoteños Amomocic
Pono
Poscueris Ríala
Pérelis
Parva
Potán
Pauján
Pulol 
----
iter 27600, loss: 44.002681
----
 tlán
San Juan Panantiag
San Juan Anucacotopan
San Jual TAlatlan Cahuiso Dímán
San Tencuantos
Lo Jitacalus Latlareno
Sans Jarzalteo Jaltoní
San Juen
San Juas Etucan
San Juan Babotzos Zidallatla
San Jua 
----
iter 27800, loss: 43.107856
----
 a
San Chatlán Ciraya
Santa Jatal Corri
Santa Crús
Santra
Santa Tertiata Vartia Ténran Chinlo
Santo Yojacona Elto Coxatlán de Cualaloras de Crús Carolta Catzana Genicuchin
San Tuparn
Santo Tepalan
Sand 
----
iter 28000, loss: 41.136994
----
  Taloel Yupeal
Tchinguchijachicutepec
Tidrebe Titrachicuarango
Tomueril de Dos Pebanatlán
Tenla
Teyapan
Tehda
Tacáno
Tejurío de Magueccora
Tepenezomo
Sacaé Terd
Samachehitlán Tlay Tlaza
Terón Tutena
C 
----
iter 28200, loss: 40.083120
----
 lla gosphutemé
Ziera Titastaco

...

----
 
Tidero Nuebanisca
Tala
Timarlichila
Santo Tocomenes
Tehualtec
Textlarad
Tlayotlán
Tlanas
Tlerrindéc
Santo Temetlán
Santo Textepeto
Tisguelol
Telasamulco de los del Prín Zambacán
Tepamalte
Tolotlapan
 
----
iter 360800, loss: 32.702600
----
 nte
Refoquezo
Amagalá
Huinoror de Péjala
Acaan
Atlalco
Altos
Huilipe
Añbal
Apalpan
Aponfilón
Acutáncache
Améy Hidérdez
Acueyalc
Ajumis
Aquel de Majapa
Youca
Zicima
Aocicha de Grarver Braccon
Acunticar 
----
iter 361000, loss: 33.777711
----
 túzela
Coxcipand
Menerado
Hezutepec
Emiria Zapaco
Cupujióndab
Cuala
Cuequingo de Coapera
Fresc. ar Aquisto de Elporco
Cuajimacutla
Conucarjo
Cuetuñadio
Huatza
Cueras Arro
Guazapan
Cuapana
Cocaco de Br 
----
iter 361200, loss: 35.079234
----
 deaspe Tezidro Aldago
Huenadro del Ros
La Martín de Tuzhuacás Huaxkara
Liuangereso
Juan Yutincuán
Tiscapacas
Lagro
Huetara de Juan Huanguadro
Jorico
Dzipngaz
Ixcapan
Osonic
Bustérdez
Loradas
Fronger d 
----
iter 361400, loss: 35.804618
----
 Las
San Juan Ello Ag,redal

----
 cáo de Solenelo
San Bartepec
Santa Inésisco
Santa Yegacaso lol
Santa Chunibla choc
Santiago de los Purtian
Santa Anerde
Santos
Santistiagón
Santa Lucía Saucuera
San Pide
Sanaltla de Juan Lánatepec
San 
----
iter 367600, loss: 33.553537
----
 tingo Teutlán
Singo
Santo Lachulco
Quema
Santoa
Santor Minatepec
Santo Tamagotlayucec
Santos
Quimaza
Santo Tamagutando
Santo Toutisda Calpan
Tistaxatlán
Santo TBatoquijeya
Sanaó
Jouzina Huatlan
Santos 
----
iter 367800, loss: 32.233144
----
 ded Beta
Trosul
Suchopan
Villa Chérez
Quijuerencispan
Victoria
ArantOcorian
Villo
Tanzama Vejisé
Santa María Moric Gro
Zapautlán de Martín
Tomaya
yus
Xoutlán de Xixtlax
Viltoliantan
Villa Cosaretepec  
----
iter 368000, loss: 33.027982
----
 hiapa
Chalpan
Chicamixtla
Cináínista
Chatelaán de los Vallende
Jiboniso
Chachilán
Cuihuatez
Cengza
Chárazápam
Karquetelo Batachan
Huhaxca
Carrites
Certirria
Caschihualá
Chacachi
Chicucas
Amurmes
Escaz 
----
iter 368200, loss: 34.538559
----
 Nutla Genac Alavejes
Huini

----
 
San Anzihuegos Xinimín
San Atitepec de Roa Minonadalda
Metlautla
Acoqualón
San Avdamtla de Cuntzintlatztlan
Vallendo Cartonge de la
San Antontono
San Jos
San Andrés Otepec
San Andan de Cabre
San Apb  
----
iter 374400, loss: 36.543392
----
 uel erdo
San Miguel Hhuatla
San Miguel Blaltilenaneerapandisto Mazón
Buedredez
San Marancio Mayguchuacá
San Miguel Chotenisto
San Miguel Tlacuerjo
San Miguel Alto
San Midalestipec os
San Miguel de Ram 
----
iter 374600, loss: 34.611374
----
 o Cocotlván
Floana de Flpajerrel
Santa María Antolla
Santa Maria Alerfoso
Santa María Zomastitás Reyes Quituaciatlán
Santiago Yauhtama
Santa María Yedalpan
Santa María Yalto
Rosilibli
Santa Martín Luc 
----
iter 374800, loss: 32.407435
----
 pan
Tizaliltitzinis de Esjoringo de Tepagáí
Esco
Tepetlán de Reforgoravee Tuquíc
Tencuntlán
Tlapala
Tlalapalaza
Tamojuelupe
Tepemillor
Tlalco
Tecutla
Texucolchic
Tlatepac
Tixtepec
Teka
Taquichulaxca
T 
----
iter 375000, loss: 32.609614
----
 e Galeo Yocucan
Zacupan
Ay

----
 n
Huquico Luceengoso Carotel Rimilán
Huarquéáp Gualán Disto
Huapanaca de la Loro
Huzuztenger
Huahuayap
Guerre Ferringando
Huauxhía de Romíro Textivo
Huembagui de Hidalgayue de Cordamo Guerrero
Hidalie 
----
iter 381200, loss: 35.532439
----
 es
Madero
Mantitec
Santiago Yálenco delislo del Puinco
Nulandillado Coaz
Nodres
Nacayine Tomás Atlista Comínepe Vera
Mlillo
Loo de Bravó
Migoral de Marchinos de Juárez
Morda
Nemámáé
Nranca
Gardado
Ros 
----
iter 381400, loss: 36.125426
----
  Coajontzos Felacío Lauháoyo
San Juinilla
San Juan Coatrininicisco Pacertan
Sontepandagón Gaancoxamitla de Cuicajojajayotlín
San Juan Mixtite
San Juan Carcó
Sucgotepec
San Juan Yejagos zás Cringo Cote 
----
iter 381600, loss: 35.694966
----
 o Laqueltla
Santa Anacoturies
Santa Catlaltepec
Santa Cruíciscdid de Otema
Santuocomi
Santa Ana
San Bartepek Sozec
San Vicentla zos
San Salvado
Santca
Silos
Santos
San Sebles
Santos del Galetenes
San  
----
iter 381800, loss: 33.491292
----
 ca
Soltlamitltán
Santo Ixt

KeyboardInterrupt: 